### **2주차 프로젝트**

이번 프로젝트에서는 Upbit API를 활용해 암호화폐의 시세 데이터를 수집하고,
이를 전처리 → 데이터베이스 저장 → 시각화까지 연결하는 데이터 파이프라인 전체 흐름을 구현합니다.

과제는 **필수 과제**와 **도전 과제**로 나뉘며, 필수 과제는 반드시 제출해야 하고, 도전 과제는 선택적으로 수행하면 됩니다.

---

**[필수 과제]**
*   문제1. 종가와 시가의 차이 계산
*   문제2. 5일 이동평균선 계산 및 결측치 처리

**[도전 과제]**
*   문제3. 종가와 5일 이동평균선 시각화

In [2]:
# 필요한 라이브러리 설치 (주석 해제 후 실행)
!pip install pyupbit pyjwt pandas matplotlib


  Using cached pyupbit-0.2.34-py3-none-any.whl.metadata (15 kB)


In [3]:
import pyupbit
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime, timedelta
import time
import warnings

# 경고 메시지 숨기기
warnings.filterwarnings('ignore')

# 한글 폰트 설정
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False



# Upbit API 클래스 구현


In [17]:
pyupbit.get_current_price(["KRW-BTC"])

171049000.0

In [13]:
class UpbitDataCollector:
    """Upbit API를 활용한 데이터 수집 클래스"""

    def __init__(self):
        self.supported_tickers = None

    def get_krw_tickers(self):
        """KRW 마켓의 모든 티커 조회"""
        try:
            tickers = pyupbit.get_tickers(fiat="KRW")
            self.supported_tickers = tickers
            return tickers
        except Exception as e:
            print(f"티커 조회 오류: {e}")
            return []

    def get_current_prices(self, tickers):
        """현재가 조회"""
        try:
            if isinstance(tickers, str):
                return pyupbit.get_current_price(tickers)
            else:
                return pyupbit.get_current_price(tickers)
        except Exception as e:
            print(f"현재가 조회 오류: {e}")
            return None

    def get_ohlcv_data(self, ticker, interval="day", count=30):
        """OHLCV 데이터 조회"""
        try:
            df = pyupbit.get_ohlcv(ticker, interval=interval, count=count)
            if df is not None:
                df['ticker'] = ticker
                df.reset_index(inplace=True)
            return df
        except Exception as e:
            print(f"{ticker} OHLCV 데이터 조회 오류: {e}")
            return None

    def get_multiple_ohlcv(self, tickers, interval="day", count=30, delay=0.1):
        """여러 티커의 OHLCV 데이터 일괄 조회"""
        all_data = []

        for ticker in tickers:
            print(f"{ticker} 데이터 수집 중...")
            data = self.get_ohlcv_data(ticker, interval, count)
            if data is not None:
                all_data.append(data)
            time.sleep(delay)  # API 호출 제한 고려

        if all_data:
            return pd.concat(all_data, ignore_index=True)
        return pd.DataFrame()

# 클래스 인스턴스 생성
collector = UpbitDataCollector()



# 데이터 수집


In [22]:
major_tickers = "KRW-BTC" #["KRW-BTC","KRW-ETC","KRW-XRP"]

# 현재가 조회
current_prices = collector.get_current_prices(major_tickers)
current_prices

171053000.0

In [23]:
major_tickers = ["KRW-BTC","KRW-ETC","KRW-XRP"]

# 현재가 조회
current_prices = collector.get_current_prices(major_tickers)
current_prices

{'KRW-BTC': 171070000.0, 'KRW-ETC': 24560.0, 'KRW-XRP': 3666.0}

In [24]:
def collect_market_data():
    """시장 데이터 수집 함수"""


    # 주요 암호화폐 선택
    major_tickers = ["KRW-BTC","KRW-ETC","KRW-XRP"]
    print(f"\n분석 대상 티커: {major_tickers}")

    # 현재가 조회
    current_prices = collector.get_current_prices(major_tickers)
    print(f"\n현재가 정보:")
    for ticker, price in current_prices.items():
        print(f"{ticker}: {price:,}원")

    # OHLCV 데이터 수집 (최근 30일)
    print(f"\nOHLCV 데이터 수집 중...")
    ohlcv_data = collector.get_multiple_ohlcv(major_tickers, count=30)

    print(f"수집된 데이터 행 수: {len(ohlcv_data)}")
    print(f"데이터 컬럼: {list(ohlcv_data.columns)}")

    return ohlcv_data, current_prices

# 데이터 수집 실행
raw_data, prices = collect_market_data()



분석 대상 티커: ['KRW-BTC', 'KRW-ETC', 'KRW-XRP']

현재가 정보:
KRW-BTC: 171,199,000.0원
KRW-ETC: 24,470.0원
KRW-XRP: 3,664.0원

OHLCV 데이터 수집 중...
KRW-BTC 데이터 수집 중...
KRW-ETC 데이터 수집 중...
KRW-XRP 데이터 수집 중...
수집된 데이터 행 수: 90
데이터 컬럼: ['index', 'open', 'high', 'low', 'close', 'volume', 'value', 'ticker']


In [6]:
# 수집된 데이터 미리보기
print(f"\n데이터 형태: {raw_data.shape}")
print(f"\n데이터 타입:\n{raw_data.dtypes}")



데이터 형태: (90, 8)

데이터 타입:
index     datetime64[ns]
open             float64
high             float64
low              float64
close            float64
volume           float64
value            float64
ticker            object
dtype: object


In [40]:
raw_data.head(10)

,index,open,high,low,close,volume,value,ticker
0,2025-09-12 09:00:00,160395000.0,161794000.0,160000000.0,161101000.0,1460.879412,2.347352e+11,KRW-BTC
1,2025-09-13 09:00:00,161110000.0,161369000.0,159461000.0,160536000.0,1011.504879,1.623566e+11,KRW-BTC
2,2025-09-14 09:00:00,160536000.0,161200000.0,159899000.0,160338000.0,734.226337,1.178583e+11,KRW-BTC
3,2025-09-15 09:00:00,160338000.0,161791000.0,159500000.0,160705000.0,1328.264880,2.131250e+11,KRW-BTC
4,2025-09-16 09:00:00,160705000.0,162334000.0,160125000.0,162167000.0,1013.134977,1.633157e+11,KRW-BTC
5,2025-09-17 09:00:00,162200000.0,163000000.0,160210000.0,161468000.0,1630.740767,2.638930e+11,KRW-BTC
6,2025-09-18 09:00:00,161468000.0,163333000.0,161372000.0,162478000.0,1738.783457,2.829548e+11,KRW-BTC
7,2025-09-19 09:00:00,162478000.0,163080000.0,161500000.0,162138000.0,1389.246280,2.256612e+11,KRW-BTC
8,2025-09-20 09:00:00,162162000.0,162486000.0,162000000.0,162236000.0,694.365420,1.126859e+11,KRW-BTC
9,2025-09-21 09:00:00,162236000.0,162239000.0,161372000.0,161412000.0,956.397868,1.547501e+11,KRW-BTC


In [30]:
df = raw_data.head(5)

In [43]:
for key, each_df in  raw_data.groupby('ticker'):
    print(key)
    display(each_df.head(2))

KRW-BTC


,index,open,high,low,close,volume,value,ticker
0,2025-09-12 09:00:00,160395000.0,161794000.0,160000000.0,161101000.0,1460.879412,2.347352e+11,KRW-BTC
1,2025-09-13 09:00:00,161110000.0,161369000.0,159461000.0,160536000.0,1011.504879,1.623566e+11,KRW-BTC


KRW-ETC


,index,open,high,low,close,volume,value,ticker
30,2025-09-12 09:00:00,29780.0,30560.0,29650.0,30510.0,574263.041878,1.723145e+10,KRW-ETC
31,2025-09-13 09:00:00,30510.0,31110.0,30190.0,30590.0,658680.100116,2.019188e+10,KRW-ETC


KRW-XRP


,index,open,high,low,close,volume,value,ticker
60,2025-09-12 09:00:00,4224.0,4343.0,4210.0,4312.0,1.085383e+08,4.625051e+11,KRW-XRP
61,2025-09-13 09:00:00,4311.0,4412.0,4282.0,4324.0,1.202434e+08,5.221643e+11,KRW-XRP


# 데이터 전처리

## [필수 과제] 문제1: 종가와 시가의 차이 계산
종가와 시가의 차이를 계산하여 price_change 컬럼에 저장하시오


### 📌 구현 단계
1. **`price_change` 컬럼 생성**  
   - `close`(종가) - `open`(시가)의 차이를 계산하여 새로운 컬럼으로 추가한다.
2. **`price_change_pct` 컬럼 계산**
   - `price_change / open * 100`으로 변동률(%)을 계산한다.
3. **`high_low_diff` 컬럼 추가**
   - `high - low`를 계산하여 일중 변동폭을 나타내는 컬럼을 만든다.



<details>
<summary>💡 구현 힌트 (펼쳐보기)</summary>

- DataFrame의 컬럼 간 계산은 바로 가능합니다.
  ```python
  df['price_change'] = df['close'] - df['open']
  df['price_change_pct'] = (df['price_change'] / df['open']) * 100

- 추가 컬럼은 반드시 원본 데이터프레임에 병합합니다.
</details>  

<br>  

---
<br>  
  
## [필수 과제] 문제2: 5일 이동평균선 계산 및 결측치 처리
5일 이동평균선 값을 계산하여 'ma5' 컬럼에 저장하고 결측치를 처리하시오


### 📌 구현 단계
1. **이동평균선(`ma5`) 계산**  
   - 각 코인(`ticker`)별로 `close` 기준 5일 이동평균을 계산한다.
2. **`결측치 처리**
   - 이동평균 계산 초기에 생기는 NaN을 `close` 값으로 대체한다.
3. **전처리 완료 후 컬럼 순서 정리**
   - `ma5` 컬럼을 포함하여 최종 순서를 정돈한다.



<details>
<summary>💡 구현 힌트 (펼쳐보기)</summary>

- 이동평균은 **rolling() 메서드**를 활용할 수 있습니다.
- 티커별로 계산하려면 **조건 필터링(mask)** 또는 **groupby()** 로 나누어 수행합니다.
- 결측치 보정 시에는 **fillna()** 메서드를 활용해 보세요.
- 이동평균을 추가한 후에는 **데이터프레임 컬럼 순서**를 다시 맞춰줍니다.

In [ ]:
def preprocess_data(df):
    """데이터 전처리 함수"""
    print("=== 데이터 전처리 시작 ===")

    # 데이터 복사
    processed_df = df.copy()

    # 날짜 컬럼 처리
    if 'index' in processed_df.columns:
        processed_df['date'] = processed_df['index']
        processed_df.drop('index', axis=1, inplace=True)

    # 날짜를 문자열로 변환 (DB 저장용)
    processed_df['date_str'] = processed_df['date'].dt.strftime('%Y-%m-%d %H:%M:%S')

    # 기술적 지표 계산

    ##############################################################
    #### 문제1) 종가와 시가의 차이 계산의 답안을 작성해주세요####




    #### 문제2) 5일 이동평균선 계산 및 결측치 처리의 답안을 작성해주세



    ##############################################################

    # 컬럼 순서 정리
    columns_order = ['date', 'date_str', 'ticker', 'open', 'high', 'low', 'close', 'volume',
                    'price_change', 'price_change_pct', 'high_low_diff', 'ma5']
    processed_df = processed_df[columns_order]

    print(f"전처리 완료 - 행 수: {len(processed_df)}")
    print(f"추가된 컬럼: price_change, price_change_pct, high_low_diff, ma5")

    return processed_df

# 데이터 전처리 실행
processed_data = preprocess_data(raw_data)


In [ ]:
processed_data.head(10)

# 데이터베이스 저장


In [ ]:
def create_database():
    """SQLite 데이터베이스 생성 함수"""
    conn = sqlite3.connect('crypto_data.db')
    cursor = conn.cursor()

    return conn

def save_to_database(df):
    """데이터베이스에 데이터 저장 함수"""
    print("=== 데이터베이스 저장 시작 ===")

    conn = create_database()

    # 데이터 저장용 DataFrame 준비
    db_df = df[['date_str', 'ticker', 'open', 'high', 'low', 'close', 'volume',
               'price_change', 'price_change_pct', 'high_low_diff', 'ma5']].copy()

    # 데이터베이스에 저장
    db_df.to_sql('crypto_ohlcv', conn, if_exists='replace', index=False)

    # 저장된 데이터 확인
    cursor = conn.cursor()
    cursor.execute("SELECT COUNT(*) FROM crypto_ohlcv")
    count = cursor.fetchone()[0]

    print(f"데이터베이스에 {count}개 레코드 저장 완료")

    conn.close()
    return count

def load_from_database():
    """데이터베이스에서 데이터 로드 함수"""
    conn = sqlite3.connect('crypto_data.db')

    query = "SELECT * FROM crypto_ohlcv ORDER BY date_str, ticker"
    df = pd.read_sql_query(query, conn)

    conn.close()
    return df

# 데이터베이스 저장 실행
saved_count = save_to_database(processed_data)

# 저장된 데이터 확인
db_data = load_from_database()
print(f"\n데이터베이스에서 로드된 데이터: {len(db_data)}개")
print(db_data.head())


# 데이터 시각화

## [도전 과제] 문제3: 종가와 5일 이동평균선 시각화
종가와 5일 이동평균선을 시각화 하시오.

### 📌 구현 단계
1. **시각화 데이터 준비**  
   - `date` 컬럼을 `datetime` 형식으로 변환
   - 티커별 데이터를 정렬 후 차트로 표현
2. **차트 작성**
   - x축: 날짜, y축: 가격
   - 라인1: 종가(close)
   - 라인2: 5일 이동평균(ma5)
3. **그래프 설정**
   - 제목, 축 라벨, 범례 추가
   - 날짜 포맷(`%m-%d`) 지정



<details>
<summary>💡 구현 힌트 (펼쳐보기)</summary>

- x축 날짜를 보기 좋게 표시하려면 **`mdates.DateFormatter`** 를 활용할 수 있습니다.
- 두 선의 구분을 위해 **라인 스타일(linestyle)** 이나 **투명도(alpha)** 를 조정해 보세요.
- 3개 암호화폐를 한 화면에 배치하고 싶다면,
**서브플롯(subplot)** 구조를 활용하면 좋습니다.
</details>  

In [ ]:
def plot_price_trends(df):
    """가격 트렌드 시각화 함수"""
    print("=== 가격 트렌드 시각화 ===")

    # 날짜 컬럼을 datetime으로 변환
    df['date'] = pd.to_datetime(df['date_str'])

    # 고유 티커 목록
    tickers = df['ticker'].unique()

    # 서브플롯 생성
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    axes = axes.flatten()

    for i, ticker in enumerate(tickers):

        ticker_data = df[df['ticker'] == ticker].sort_values('date')

        ax = axes[i]

        ##############################################################
        #### 문제3) 종가와 5일 이동평균선 시각화의 답안을 작성해주세요 ####




        ##############################################################

        # 날짜 포맷 설정
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%m-%d'))
        ax.tick_params(axis='x', rotation=45)

    plt.show()



# 시각화 실행
plot_price_trends(db_data)


In [ ]:
plt.plot()